In [16]:
import numpy as np
import torch

data = torch.load("preprocessed_images_with_metadata_and_target.pt.nosync")

In [17]:
features = []
metadata = []
y = []

for [target, age, implant], image in data:
    features.append(image.numpy())
    metadata.append((age, implant))
    y.append(target)


In [18]:
features = torch.tensor(np.array(features).reshape((-1, 1, 512, 512))).float()
metadata = torch.tensor(np.asarray(metadata).reshape((-1, 2))).float()
y = torch.tensor(y).view(-1, 1).float()

In [20]:
features.shape

torch.Size([50, 1, 512, 512])

In [19]:
metadata.shape

torch.Size([50, 2])

In [6]:
y

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]])

In [21]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(features, metadata, y, test_size=0.15,
                                                                      random_state=3451)

train_dataset = TensorDataset(torch.Tensor(np.asarray(X1_train)).float(), X2_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(torch.Tensor(np.asarray(X1_val)).float(), X2_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [26]:
from torchvision.models import EfficientNet_B2_Weights
from torchvision import models
import torch.nn as nn


class EfficientNetGrayWithoutClassifier(nn.Module):
    def __init__(self):
        super(EfficientNetGrayWithoutClassifier, self).__init__()
        self.base_model = models.efficientnet_b2(weights=EfficientNet_B2_Weights.DEFAULT)
        self.base_model.classifier = nn.Sequential(
            nn.Linear(1408, 512),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(512, 64),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Dropout(0.3),
            nn.Sigmoid()
        )
        self.seq_2 = nn.Sequential(
            nn.Linear(2, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
            nn.Sigmoid()
        )
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        self.base_model.eval()

    def forward(self, single_channel_input, metadata):
        rgb_input = torch.cat([single_channel_input] * 3, dim=1)
        out_1 = self.base_model(rgb_input)
        out_2 = self.seq_2(metadata)

        out = torch.cat([out_1, out_2], dim=1)
        out = self.linear(out)
        out = self.sigmoid(out)

        return out




In [27]:
def probabilistic_f1(labels, predictions, beta=1):
    pTP = np.sum(labels * predictions)
    pFP = np.sum((1 - labels) * predictions)
    num_positives = np.sum(labels)  #  = pTP+pFN

    pPrecision = pTP / (pTP + pFP)
    pRecall = pTP / num_positives

    beta_squared = beta ** 2

    if pPrecision > 0 and pRecall > 0:
        pF1 = (1 + beta_squared) * pPrecision * pRecall / (beta_squared * pPrecision + pRecall)
        return pF1
    else:
        return 0

In [28]:
results = {}

In [29]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_auc_score
from matplotlib import pyplot as plt
from torch import optim

model = EfficientNetGrayWithoutClassifier()

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 100

train_losses = []
val_losses = []
train_f1_scores = []
val_f1_scores = []

for epoch in range(num_epochs):
    plt.clf()
    running_loss = 0.0
    model.train()
    train_predictions = []
    train_labels = []

    for i, data in enumerate(train_dataloader, 0):
        inputs1, inputs2, labels = data

        optimizer.zero_grad()

        outputs = model(inputs1, inputs2)
        loss = criterion(outputs, labels)

        train_predictions.extend(outputs.detach().numpy().flatten())
        train_labels.extend(labels.detach().numpy().flatten())

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / (i + 1)
    train_losses.append(train_loss)
    train_f1 = probabilistic_f1(np.asarray(train_labels), np.round(train_predictions, decimals=2))
    train_f1_scores.append(train_f1)

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for i, data in enumerate(val_dataloader, 0):
            inputs1, inputs2, labels = data

            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, labels)

            val_predictions.extend(outputs.detach().numpy().flatten())
            val_labels.extend(labels.detach().numpy().flatten())

            val_loss += loss.item()

    val_loss = val_loss / (i + 1)
    val_losses.append(val_loss)
    val_f1 = probabilistic_f1(np.asarray(val_labels), np.asarray(val_predictions))
    val_f1_scores.append(val_f1)

    print(
        f'Epoch {epoch + 1}, Validation Loss: {round(val_loss, 3)}, F1 Score: {round(val_f1, 3)} Training Loss: {round(train_loss, 3)}, F1 Score: {round(train_f1, 3)}')

# Real-time visualization
plt.figure(1)
plt.plot(train_losses, label='Training Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.title('Training and Validation Loss')

plt.figure(2)
plt.plot(train_f1_scores, label='Training F1 Score', color='green')
plt.plot(val_f1_scores, label='Validation F1 Score', color='orange')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend(loc='upper left')
plt.title('Training and Validation F1 Score')

plt.show(block=False)

y_pred = np.asarray(val_predictions) > 0.5
accuracy = accuracy_score(val_labels, y_pred)
precision = precision_score(val_labels, y_pred, average='weighted')
recall = recall_score(val_labels, y_pred, average='weighted')
auroc = roc_auc_score(val_labels, val_predictions, multi_class='ovr')

clf_name = "DeepNN - val"
results[clf_name] = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': val_f1,
    'AUROC': auroc,
}
y_pred = np.asarray(train_predictions) > 0.5
accuracy = accuracy_score(train_labels, y_pred)
precision = precision_score(train_labels, y_pred, average='weighted')
recall = recall_score(train_labels, y_pred, average='weighted')
auroc = roc_auc_score(train_labels, train_predictions, multi_class='ovr')

clf_name = "DeepNN - train"
results[clf_name] = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': train_f1,
    'AUROC': auroc,
}


Epoch 1, Validation Loss: 0.693, F1 Score: 0.496 Training Loss: 0.694, F1 Score: 0.517
Epoch 2, Validation Loss: 0.693, F1 Score: 0.496 Training Loss: 0.694, F1 Score: 0.517
Epoch 3, Validation Loss: 0.693, F1 Score: 0.497 Training Loss: 0.693, F1 Score: 0.52



KeyboardInterrupt



<Figure size 640x480 with 0 Axes>